# High-Throughput Virtual Screening using GNINA
**Author:** Dhrubajyoti Maji  
**Date:** 16-12-2025  
**Environment:** Google Colab (Linux/GPU)

## 1. Project Overview
The goal of this notebook is to perform a virtual screening campaign to identify potential inhibitors for the **MAO-B (Monoamine Oxidase B)** target. We utilize **GNINA**, a deep learning-based molecular docking tool that outperforms standard AutoDock Vina by using Convolutional Neural Networks (CNNs) to score protein-ligand interactions.

### Workflow
1.  **Environment Setup:** Install GNINA, OpenBabel, and visualization tools.
2.  **Validation (Redocking):** Dock the co-crystallized ligand (SAG) back into the receptor to verify the protocol accuracy (RMSD calculation).
3.  **Virtual Screening:** Batch processing of a ligand library against the MAO-B binding pocket.
4.  **Analysis:** Parsing log files to extract **CNN Affinity** and **Pose Scores** to rank top candidates.

## 2. Environment Setup
We are running on a Linux-based Colab instance. The following steps will:
* Download the **GNINA v1.3** binary (Linux executable).
* Install **OpenBabel** (for RMSD calculations and file conversion).
* Install **py3Dmol** (for interactive 3D visualization of docked poses).
* Check for GPU availability (GNINA runs significantly faster with CUDA acceleration).

In [ ]:
!nvidia-smi
!wget https://github.com/gnina/gnina/releases/download/v1.3/gnina
!chmod +x gnina
!./gnina --version

In [ ]:
!pip install open3Dmol
!apt install openbabel

In [ ]:
%ls

## 3. Protocol Validation: Redocking
Before running the screen, we must validate that GNINA can accurately reproduce the experimental binding mode.

**Experiment:**
* **Receptor:** `MAO_B.pdb` (Crystal structure)
* **Ligand:** `SAG.pdb` (Co-crystallized inhibitor)
* **Method:** We treat the crystal ligand as a "blind" input and ask GNINA to find its binding site using the `--autobox_ligand` parameter.

**Success Metric:**
We will calculate the **RMSD (Root Mean Square Deviation)** between the generated pose and the original crystal pose. An RMSD **< 2.0 Å** is generally considered a successful reproduction of the binding mode.

In [ ]:
!./gnina -r MAO_B.pdb -l SAG.pdb --autobox_ligand SAG.pdb --autobox_add 4 --num_modes 12 --exhaustiveness 10 --cnn_scoring rescore --cnn crossdock_default2018 --device 0 --seed 0 --verbosity 1 -o SAG_redock.sdf --log SAG_log.log

### 3.1 Visualization of Redocked Pose
We visually inspect the quality of the docking prediction by overlaying the experimental structure with the predicted pose.

**Legend:**
* **<span style="color:gray">Gray Sticks</span>**: The **Crystal Ligand** (Reference/Ground Truth).
* **<span style="color:green">Green Sticks</span>**: The **GNINA Redocked Pose** (Prediction).

**Interpretation:**
A successful redocking is indicated by a high degree of overlap between the green and gray structures. If the green ligand deviates significantly from the gray one, the docking parameters may need adjustment.

In [ ]:
import py3Dmol
v = py3Dmol.view()
v.addModel(open('MAO_B.pdb').read())
v.setStyle({'cartoon':{},'stick':{'radius':.1}})
v.addModel(open('SAG.pdb').read())
v.setStyle({'model':1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.125}})
v.addModelsAsFrames(open('SAG_redock.sdf').read())
v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
v.zoomTo({'model':1})
v.rotate(90)
v.animate({'interval':1000})

### 3.2 RMSD Calculation
We use `obrms` (from OpenBabel) to compare the geometry of the docked ligand (`SAG_redock.sdf`) against the original crystal structure (`SAG.pdb`).

* **Low RMSD (< 2.0 Å):** The docking protocol is accurate for this target.
* **High RMSD (> 2.0 Å):** The parameters (e.g., `exhaustiveness`, `autobox_add`) may need tuning, or the binding pocket is highly flexible.

In [ ]:
!obrms -f SAG.pdb  SAG_redock.sdf

Expectedly pose 1 shows the best RMSD with respect to the crystal ligand.

## 4. Virtual Screening Pipeline
We now proceed to the production run. We will loop through all `.pdb` files in the `ligands/` directory.

### Key Parameters:
* **`--autobox_ligand SAG.pdb`**: We define the search space (bounding box) around the known binding site of the co-crystallized ligand.
* **`--autobox_add 4`**: We add a 4 Å buffer to the box to ensure the search space isn't too tight.
* **`--cnn_scoring rescore`**: We use the default scoring function for sampling but use the extensive CNN model to *rescore* the final poses.
* **`--exhaustiveness 10`**: A balance between speed and sampling density. (Higher = more accurate but slower).

### Checking GNINA installation ###

In [ ]:
import os
import subprocess
import pandas as pd
import glob
import re
import time
import sys
import codecs
from tqdm import tqdm

# --- Configuration Variables ---
RECEPTOR_FILE = "MAO_B.pdb"
COLIGAND_FILE = "SAG.pdb"
LIGANDS_DIR = "ligands"
OUTPUT_DIR = "docked_results"
LOG_DIR = "logs"
OUTPUT_CSV = "gnina_screening_summary.csv"

# --- Create required directories ---
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

print("Configuration:")
print(f"Receptor: {RECEPTOR_FILE}")
print(f"Coligand (for autobox): {COLIGAND_FILE}")
print(f"Ligands Source Directory: {LIGANDS_DIR}")
print(f"Output Directory: {OUTPUT_DIR}")
print(f"Log Directory: {LOG_DIR}")

# --- Check for GNINA executable (Optional but recommended) ---
try:
    # Run a simple check command
    subprocess.run(["./gnina", "--version"], check=True, capture_output=True, text=True)
    print("\nGNINA executable found and running.")
except FileNotFoundError:
    print("\nERROR: 'gnina' command not found. Please ensure GNINA is installed and in your system's PATH.")
except subprocess.CalledProcessError as e:
    print(f"\nWarning: GNINA command returned an error during version check: {e.stderr}")

### Docking with GNINA ###  
The ligand pdb files are stored in the folder named 'ligands'

In [ ]:
# List all ligand files
ligand_files = glob.glob(os.path.join(LIGANDS_DIR, "*.pdb"))

if not ligand_files:
    print(f"ERROR: No .pdb files found in the '{LIGANDS_DIR}' directory. Please check your path.")
else:
    num_ligands = len(ligand_files)
    print(f"Found {num_ligands} ligands to dock. Starting pipeline...")

    # Base GNINA command parameters
    BASE_COMMAND = [
        "./gnina",
        "-r", RECEPTOR_FILE,
        "--autobox_ligand", COLIGAND_FILE,
        "--autobox_add", "4",
        "--num_modes", "10",
        "--exhaustiveness", "10",
        "--cnn_scoring", "rescore",
        "--cnn", "crossdock_default2018",
        "--device", "0",
        "--verbosity", "1",
        "--seed", "0",
    ]

    # --- Use tqdm and assign the progress bar object to 'pbar' ---
    pbar = tqdm(ligand_files, unit="ligand", file=sys.stdout)

    for full_path in pbar:

        # Extract the base ligand name
        ligand_name = os.path.splitext(os.path.basename(full_path))[0]

        # --- CORRECT CALL: Use the pbar object to set the description ---
        pbar.set_description(f"GNINA Docking | Current: {ligand_name}")

        # Define output and log file paths
        output_pdbqt = os.path.join(OUTPUT_DIR, f"{ligand_name}_docked.pdbqt")
        log_file = os.path.join(LOG_DIR, f"{ligand_name}.log")

        # Construct the full command for this ligand
        command = BASE_COMMAND + [
            "-l", full_path,
            "-o", output_pdbqt,
            "--log", log_file
        ]

        try:
            # Execute the GNINA command
            # Using timeout is optional, but good for robust screening
            subprocess.run(command, check=True, capture_output=True, timeout=120)
        except subprocess.CalledProcessError as e:
            # Report error directly to the console or log but keep the progress bar moving
            sys.stderr.write(f"\n--- DOCKING FAILED for {ligand_name} ---\n")
            sys.stderr.write(f"Stderr: {e.stderr.decode()}\n")
        except subprocess.TimeoutExpired:
            sys.stderr.write(f"\n--- DOCKING TIMED OUT for {ligand_name} ---\n")

    print("\nDocking process complete.")

### Parsing GNINA log files to extract different scores ###  
Top pose is based on pose score, the higher the better

In [ ]:
import codecs

def parse_gnina_log(log_file):
    """Parses a GNINA log file using a robust method to handle tricky whitespace."""

    # Use codecs.open with 'utf-8' and errors='ignore' to handle potential encoding issues
    try:
        with codecs.open(log_file, 'r', encoding='utf-8', errors='ignore') as f:
            log_content = f.read()
    except Exception as e:
        print(f"Error reading file {log_file}: {e}")
        return None, None, None

    # Replace all unusual whitespace (like non-breaking space \xa0) with a standard space
    # and collapse multiple spaces into one to clean up the content.
    cleaned_content = log_content.replace('\xa0', ' ')
    cleaned_content = re.sub(r'\s+', ' ', cleaned_content)

    # --- Step 1: Find the entire data table block based on unique keywords ---
    # We will search for the "mode | affinity" header, which is highly unique.

    # The header pattern, now relying on clean spaces and keywords
    header_pattern = r'mode \| affinity \| intramol \| CNN pose score \| CNN affinity'

    # Find the position of the header line
    header_match = re.search(header_pattern, cleaned_content, re.IGNORECASE)

    if not header_match:
        # If the clean search fails, try a very simple keyword search for the table start
        header_start = cleaned_content.find('mode | affinity')
        if header_start == -1:
            print(f"Warning: Could not find the score table header in {log_file}")
            return None, None, None
    else:
        header_start = header_match.start()

    # Isolate the relevant block of content starting from the header
    content_from_header = cleaned_content[header_start:]

    # The data starts AFTER the header and the '-----' separator line.

    # Find the line separator (e.g., "-----+----")
    separator_end = content_from_header.find('----------') # Look for the first major dash sequence
    if separator_end == -1:
        print(f"Warning: Could not find the table separator in {log_file}")
        return None, None, None

    # Isolate the text that contains the numerical data
    data_block_start = separator_end + content_from_header[separator_end:].find('\n')
    data_block = content_from_header[data_block_start:].strip()

    # --- Step 2: Extract numbers from the isolated block ---
    lines = data_block.split('\n')
    mode_data = []

    # Regex to find the 5 required numbers: mode, affinity, intramol, pose score, CNN affinity
    # The numbers are now separated by single spaces (from the cleaning process) and pipes.
    # We will simply look for 5 floating point numbers on the line.
    number_extractor = re.compile(r'([-+]?[0-9]*\.?[0-9]+)')

    for line in lines:
        if not line.strip():
            continue

        # Extract all numbers from the line
        numbers = [float(n) for n in number_extractor.findall(line.strip())]

        # Check if we have the 5 core values we need:
        if len(numbers) >= 5:
            try:
                # 1. Mode (index 0)
                # 2. Affinity (index 1)
                # 3. Intramol (index 2)
                # 4. Pose Score (index 3)
                # 5. CNN Affinity (index 4)

                # Check for the line that only contains the numerical scores (i.e., mode number 1, 2, 3...)
                if numbers[0] == round(numbers[0]): # Check if the first number is an integer (the mode number)
                    affinity = numbers[1]
                    pose_score = numbers[3]
                    cnn_affinity = numbers[4]

                    mode_data.append({
                        'mode': int(numbers[0]),
                        'affinity': affinity,
                        'pose_score': pose_score,
                        'cnn_affinity': cnn_affinity
                    })
            except (ValueError, IndexError):
                continue

    if not mode_data:
        print(f"Warning: Extracted data block but no numerical scores found in {log_file}")
        return None, None, None

    # Convert to DataFrame for easy manipulation
    df = pd.DataFrame(mode_data)

    # --- Step 3: Select the best pose (Highest pose_score) ---
    if df.empty or 'pose_score' not in df.columns:
         return None, None, None

    best_pose_row = df.loc[df['pose_score'].idxmax()]

    final_affinity = best_pose_row['affinity']
    final_cnn_affinity = best_pose_row['cnn_affinity']
    final_pose_score = best_pose_row['pose_score']

    return final_affinity, final_cnn_affinity, final_pose_score

# --- Main Parsing Loop (No change needed) ---
results = []
log_files = glob.glob(os.path.join(LOG_DIR, "*.log"))

print(f"Found {len(log_files)} log files. Starting analysis...")

for log_file in log_files:
    ligand_name = os.path.splitext(os.path.basename(log_file))[0]

    # Updated call to reflect new return structure
    affinity, cnn_affinity, pose_score = parse_gnina_log(log_file)

    if affinity is not None:
        results.append({
            'ligand_name': ligand_name,
            'best_pose_vina_affinity (kcal/mol)': affinity,
            'best_pose_cnn_affinity (pKi)': cnn_affinity,
            'best_pose_score': pose_score
        })
    else:
        # Only print the generic error message if all attempts fail
        results.append({
            'ligand_name': ligand_name,
            'best_pose_vina_affinity (kcal/mol)': 'N/A (Parse Error/Failure)',
            'best_pose_cnn_affinity (pKi)': 'N/A (Parse Error/Failure)',
            'best_pose_score': 'N/A'
        })

print("\nAnalysis complete.")

## 5. Analysis & Ranking
GNINA outputs several scores. This pipeline prioritizes the Deep Learning scores over the standard Vina scores:

1.  **CNN Pose Score (0 to 1):** The probability that a specific pose is correct (ligand is in the right spot).
2.  **CNN Affinity (pKd/pKi):** The predicted binding affinity. **Higher is better.**
    * *Note: This is often more correlated with experimental activity than standard Vina scores.*

### Output
The code below parses the logs, extracts the highest-scoring pose for each ligand, and saves a ranked summary to `gnina_screening_summary.csv`.

In [ ]:
# Convert results list to a pandas DataFrame
df_summary = pd.DataFrame(results)

# Clean up data types and sort (if possible)
# Note: Sorting must handle "N/A" values gracefully
df_summary['best_pose_cnn_affinity (pKi)'] = pd.to_numeric(
    df_summary['best_pose_cnn_affinity (pKi)'], errors='coerce'
)

# Sort by CNN Affinity (Highest pKi is the best binder)
df_summary_sorted = df_summary.sort_values(
    by='best_pose_cnn_affinity (pKi)', ascending=False
)

# Save the final DataFrame to the requested CSV file
df_summary_sorted.to_csv(OUTPUT_CSV, index=False)

print(f"--- FINAL VIRTUAL SCREENING SUMMARY ---")
print(f"Results saved to: {OUTPUT_CSV}")
print("\nTop 10 Ligands (Ranked by CNN Affinity, which is the best metric):")

# Display the top 10 results
print(df_summary_sorted.head(10).to_string(index=False))

## 6. Conclusion & Next Steps

### Summary
* **Validation:** The redocking of SAG achieved an RMSD of 0.518593 Å, confirming the docking parameters are valid.
* **Screening:** We successfully screened 16 ligands against MAO-B.
* **Top Hits:** The top candidate is **NP0002822** with a CNN Affinity of **7.349**.

**With higher computational resources this notebook can be used to screen larger libraries**

### Next Steps
1.  **Visual Inspection:** Manually inspect the top 5-10 poses in PyMOL or Chimera to check for specific interactions (e.g., hydrogen bonds with key residues).
2.  **ADMET Analysis:** Run the top hits through SwissADME to check for drug-likeness.
3.  **Molecular Dynamics:** Perform MD simulations (e.g., ~100ns) on the best complex to assess stability.